# Create gene perturbtion responses

In [11]:
import pandas as pd
import pickle
import os

In [12]:
# read pickle dataframe 
df = pickle.load(open('/drive3/nphill22/projects/corsello_lab/20250525_manuscript/data/features/x-all.pkl', 'rb'))

/tmp/ipykernel_2249798/3878988169.py:2: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(open('/drive3/nphill22/projects/corsello_lab/20250525_manuscript/data/features/x-all.pkl', 'rb'))


In [13]:
# split out the XPR_ columns
xpr_cols = [col for col in df.columns if 'XPR_' in col]
xpr_df = df[xpr_cols]
other_df = df.drop(xpr_cols, axis=1)

In [14]:
# in xpr_df, just get columns where at lesat 10 values are less than -1, and at least 10 values are greater than -1
sens_xpr_df = xpr_df.loc[:, (xpr_df.apply(lambda col: (col < -1).sum() >= 10) & (xpr_df.apply(lambda col: (col > -1).sum() >= 10)))]

In [15]:
def create_response_df(depmap_id_val, col_val, name):
    return pd.DataFrame({
        "depmap_id": depmap_id_val,
        "broad_id": name,
        "LFC": col_val,
        "name": name,
        "ccle_name": depmap_id_val
    })

In [17]:
output_directory = "/drive3/nphill22/projects/corsello_lab/20250525_manuscript/data/responses/full/genetic_ko"

depmap_ids = other_df.index.tolist()
for col in sens_xpr_df.columns:
    col_vals = sens_xpr_df[col].tolist()
    name = col.split("_")[1]
    df = create_response_df(depmap_ids, col_vals, name)
    df.to_csv(os.path.join(output_directory, f"{name}.csv"), index=False)

In [18]:
# save the other_df as feature pickle
other_df.to_pickle("/drive3/nphill22/projects/corsello_lab/20250525_manuscript/data/features/x-all-without-xpr.pkl")